In [7]:
!nvidia-smi

Mon Jun  5 16:49:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
#import zipfile
#with zipfile.ZipFile("./drive/MyDrive/dataset.zip", 'r') as zip_ref:
#  zip_ref.extractall("./")

In [9]:
TEST_PATH = "/content/dataset/test"
SUBMISSION_CSV_PATH = "/content/dataset/sample_submission.csv"
SAVE_TO_CSV_PATH = "/content/drive/MyDrive/submission.csv"

TASK1_MODEL_PATH = "/content/drive/MyDrive/models_best/task1.pt"
TASK2_MODEL_PATH = "/content/drive/MyDrive/models_best/task2.pt"
TASK3_MODEL_PATH = "/content/drive/MyDrive/models_best/task3.pt"

In [10]:
import numpy as np
import pandas as pd
import csv
from PIL import Image
import numpy as np
import random
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
alphabets = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
alphabets2index = {alphabet:i for i, alphabet in enumerate(alphabets)}

cuda


In [12]:
class TaskDataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = data
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        path = f"{self.root}/{filename}"
        img = Image.open(path)
        preprocess = transforms.Compose([
            #transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        input_tensor = preprocess(img)
        input_tensor = input_tensor.to(device)
        
        if self.return_filename:
            return input_tensor, filename
        else:
            return input_tensor, label

    def __len__(self):
        return len(self.data)

In [13]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel,
                      outchannel,
                      kernel_size=3,
                      stride=stride,
                      padding=1,
                      bias=False),
            nn.BatchNorm2d(outchannel, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel,
                      outchannel,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                      bias=False),
            nn.BatchNorm2d(outchannel, track_running_stats=True))
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel,
                          outchannel,
                          kernel_size=1,
                          stride=stride,
                          bias=False),
                nn.BatchNorm2d(outchannel, track_running_stats=True))

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet34_1(nn.Module):
    def __init__(self, ResidualBlock, num_classes=62):
        super(ResNet34_1, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64, track_running_stats=True),
            nn.ReLU(),
        )
        # res18 2 2 2 2
        # res34 3 4 6 3
        self.layer1 = self.make_layer(ResidualBlock, 64, 3, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 4, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 6, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 3, stride=2)
        self.drop = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512, num_classes)
        #self.fc2 = nn.Linear(512, num_classes)
        #self.fc3 = nn.Linear(512, num_classes)
        #self.fc4 = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = nn.AdaptiveAvgPool2d(1)(x)
        x = x.view(-1, 512)
        x = self.drop(x)
        y1 = self.fc1(x)
        #y2 = self.fc2(x)
        #y3 = self.fc3(x)
        #y4 = self.fc4(x)
        return y1
    
class ResNet34_2(nn.Module):
    def __init__(self, ResidualBlock, num_classes=62):
        super(ResNet34_2, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64, track_running_stats=True),
            nn.ReLU(),
        )
        # res18 2 2 2 2
        # res34 3 4 6 3
        self.layer1 = self.make_layer(ResidualBlock, 64, 3, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 4, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 6, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 3, stride=2)
        self.drop = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512, num_classes)
        self.fc2 = nn.Linear(512, num_classes)
        #self.fc3 = nn.Linear(512, num_classes)
        #self.fc4 = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = nn.AdaptiveAvgPool2d(1)(x)
        x = x.view(-1, 512)
        x = self.drop(x)
        y1 = self.fc1(x)
        y2 = self.fc2(x)
        #y3 = self.fc3(x)
        #y4 = self.fc4(x)
        return y1, y2

class ResNet34_3(nn.Module):
    def __init__(self, ResidualBlock, num_classes=62):
        super(ResNet34_3, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64, track_running_stats=True),
            nn.ReLU(),
        )
        # res18 2 2 2 2
        # res34 3 4 6 3
        self.layer1 = self.make_layer(ResidualBlock, 64, 3, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 4, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 6, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 3, stride=2)
        self.drop = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512, num_classes)
        self.fc2 = nn.Linear(512, num_classes)
        self.fc3 = nn.Linear(512, num_classes)
        self.fc4 = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = nn.AdaptiveAvgPool2d(1)(x)
        x = x.view(-1, 512)
        x = self.drop(x)
        y1 = self.fc1(x)
        y2 = self.fc2(x)
        y3 = self.fc3(x)
        y4 = self.fc4(x)
        return y1, y2, y3, y4

In [14]:
test_data_t1 = []
test_data_t2 = []
test_data_t3 = []

with open(SUBMISSION_CSV_PATH, newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if row[0].startswith("task1"):
            test_data_t1.append(row)
        
        if row[0].startswith("task2"):
            test_data_t2.append(row)
        
        if row[0].startswith("task3"):
            test_data_t3.append(row)
            

print(len(test_data_t1))
print(len(test_data_t2))
print(len(test_data_t3))

6500
2500
1000


In [15]:
all_files = []
all_preds = []

test_ds = TaskDataset(test_data_t1, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)
model = ResNet34_1(ResidualBlock).to(device)
model.load_state_dict(torch.load(TASK1_MODEL_PATH))
model.eval()

for image, filenames in test_dl:
    image = image.to(device)
    pred_1 = model(image)
    pred_1 = torch.argmax(pred_1, dim=1)
    
    for i in range(len(filenames)):
        pred = alphabets[pred_1[i].item()]
        all_files.append(filenames[i])
        all_preds.append(pred)

print("write task 1 done.")

        
test_ds = TaskDataset(test_data_t2, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)        
model = ResNet34_2(ResidualBlock).to(device)
model.load_state_dict(torch.load(TASK2_MODEL_PATH))
model.eval()

for image, filenames in test_dl:
    image = image.to(device)
    pred_1, pred_2 = model(image)
    pred_1 = torch.argmax(pred_1, dim=1)
    pred_2 = torch.argmax(pred_2, dim=1)
    
    for i in range(len(filenames)):
        pred = alphabets[pred_1[i].item()]+alphabets[pred_2[i].item()]
        all_files.append(filenames[i])
        all_preds.append(pred)
        
print("write task 2 done.")

        
test_ds = TaskDataset(test_data_t3, root=TEST_PATH, return_filename=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)        
model = ResNet34_3(ResidualBlock).to(device)
model.load_state_dict(torch.load(TASK3_MODEL_PATH))
model.eval()

for image, filenames in test_dl:
    image = image.to(device)
    pred_1, pred_2, pred_3, pred_4 = model(image)
    pred_1 = torch.argmax(pred_1, dim=1)
    pred_2 = torch.argmax(pred_2, dim=1)
    pred_3 = torch.argmax(pred_3, dim=1)
    pred_4 = torch.argmax(pred_4, dim=1)
    
    for i in range(len(filenames)):
        pred = alphabets[pred_1[i].item()]+alphabets[pred_2[i].item()]+alphabets[pred_3[i].item()]+alphabets[pred_4[i].item()]
        all_files.append(filenames[i])
        all_preds.append(pred)

print("write task 3 done.")

df = pd.DataFrame(list(zip(all_files, all_preds)), columns=['filename', 'label'])
df.to_csv(SAVE_TO_CSV_PATH, index=False)

write task 1 done.
write task 2 done.
write task 3 done.
